In [1]:
import pandas as pd
import numpy as np # these two packages used mainly to preprocess data
from datetime import datetime # to convert time format

In [2]:
# Daten einlesen
opsdf=pd.read_csv(r"origData\enthält_behandlung.csv", sep=";")
icddf=pd.read_csv(r"origData\enthält_diagnose.csv", sep=";")
falldf=pd.read_csv(r"origData\fall.csv", sep=";")
patientdf = pd.read_csv(r"origData\patient.csv",sep=";")

In [3]:
# merge icddf and opsdf
icdopsdf = pd.merge(icddf,opsdf,on='Fall_ID',how='outer')
icdopsdf = icdopsdf.dropna()

In [4]:
# adapt only important column from falldf, then name it falldf_short
falldf_short=falldf.filter(items=["Fall_ID","Pat_ID","Entlassungsgrund","Verweildauer","Aufnahmedatum","Entlassungsdatum"])

In [5]:
# merge icdopsdf with falldf_short, name it fallicdopsdf, filter out unnecessary columns, rename Datum to OPS_Datum
fallicdopsdf = pd.merge(icdopsdf,falldf_short,on='Fall_ID',how='outer')
fallicdopsdf = fallicdopsdf.dropna()
fallicdopsdf=fallicdopsdf.filter(items=["Fall_ID","Pat_ID","ICD","Diagnoseart","OPS","Datum","Entlassungsgrund","Verweildauer","Aufnahmedatum","Entlassungsdatum"])
fallicdopsdf=fallicdopsdf.rename(columns={"Datum":"OPS_Datum"})

In [6]:
# Extract the low-level (more specific) from high-level (more general)
fallicdopsdf["ICDlabel"]=fallicdopsdf['ICD'].str.replace(r'(?<=[A-Z]\d{2}\.\d).*', '', regex=True) # back up plan
fallicdopsdf["ICDlabel"]=fallicdopsdf['ICDlabel'].str.replace(r'(?<=[A-Z]\d{1}\.\d).*', '', regex=True) # back up plan, we use ICD as whole for more precise result
fallicdopsdf["OPSlabel"]=fallicdopsdf['OPS'].str.replace(r'\..*', '', regex=True)

In [7]:
# patientdf reduced and merge with fallicdopsdf
patientdf.drop(['IK_Kasse','VersichertenID','PLZ'], axis=1, inplace=True)
fallpatienticdopsdf = pd.merge(fallicdopsdf,patientdf,on='Pat_ID',how='left')
fallpatienticdopsdf = fallpatienticdopsdf.dropna()
fallpatienticdopsdf

,Fall_ID,Pat_ID,ICD,Diagnoseart,OPS,OPS_Datum,Entlassungsgrund,Verweildauer,Aufnahmedatum,Entlassungsdatum,ICDlabel,OPSlabel,Geschlecht,Geburtsjahr
0,2198995,709117,P27.1,ND,1-207.0,2008-07-11 13:18:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,P27.1,1-207,w,2008
1,2198995,709117,P27.1,ND,1-207.0,2008-07-28 11:03:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,P27.1,1-207,w,2008
2,2198995,709117,P27.1,ND,1-207.0,2008-08-26 13:09:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,P27.1,1-207,w,2008
3,2198995,709117,P27.1,ND,1-207.0,2008-10-07 16:47:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,P27.1,1-207,w,2008
4,2198995,709117,P27.1,ND,1-207.0,2009-01-08 09:42:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,P27.1,1-207,w,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616932,4122526,213945,Z95.1,ND,8-930,2015-06-28 22:59:00,19,708,2015-06-28 22:59:00,2015-06-29 10:47:00,Z95.1,8-930,m,1932
1616933,4122526,213945,Z95.5,ND,8-930,2015-06-28 22:59:00,19,708,2015-06-28 22:59:00,2015-06-29 10:47:00,Z95.5,8-930,m,1932
1616934,4123258,203628,I10.00,ND,8-930,2015-06-30 07:11:00,41,427,2015-06-30 07:11:00,2015-06-30 14:18:00,I10.0,8-930,m,1958
1616935,4123258,203628,I20.9,HD,8-930,2015-06-30 07:11:00,41,427,2015-06-30 07:11:00,2015-06-30 14:18:00,I20.9,8-930,m,1958


In [8]:
# only main diagnosis
hdgesamt = fallpatienticdopsdf[fallpatienticdopsdf["Diagnoseart"]=="HD"]

In [9]:
# time reduction to year and age binning
hdgesamt['OPS_Jahr'] = pd.DatetimeIndex(hdgesamt['OPS_Datum']).year
hdgesamt['OPS_Alter'] = hdgesamt['OPS_Jahr'] - hdgesamt['Geburtsjahr']
hdgesamt['OPS_Altergruppe'] = hdgesamt['OPS_Alter']
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 0) & (hdgesamt['OPS_Altergruppe'] <= 2), ['OPS_Altergruppe']] = 0
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 3) & (hdgesamt['OPS_Altergruppe'] <= 5), ['OPS_Altergruppe']] = 1
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 6) & (hdgesamt['OPS_Altergruppe'] <= 13), ['OPS_Altergruppe']] = 2
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 14) & (hdgesamt['OPS_Altergruppe'] <= 18), ['OPS_Altergruppe']] = 3
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 19) & (hdgesamt['OPS_Altergruppe'] <= 33), ['OPS_Altergruppe']] = 4
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 34) & (hdgesamt['OPS_Altergruppe'] <= 48), ['OPS_Altergruppe']] = 5
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 49) & (hdgesamt['OPS_Altergruppe'] <= 64), ['OPS_Altergruppe']] = 6
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 65) & (hdgesamt['OPS_Altergruppe'] <= 78), ['OPS_Altergruppe']] = 7
hdgesamt.loc[(hdgesamt['OPS_Altergruppe'] >= 79), ['OPS_Altergruppe']] = 8



C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\1543531534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdgesamt['OPS_Jahr'] = pd.DatetimeIndex(hdgesamt['OPS_Datum']).year
C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\1543531534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdgesamt['OPS_Alter'] = hdgesamt['OPS_Jahr'] - hdgesamt['Geburtsjahr']
C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\1543531534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [10]:
hdgesamt

,Fall_ID,Pat_ID,ICD,Diagnoseart,OPS,OPS_Datum,Entlassungsgrund,Verweildauer,Aufnahmedatum,Entlassungsdatum,ICDlabel,OPSlabel,Geschlecht,Geburtsjahr,OPS_Jahr,OPS_Alter,OPS_Altergruppe
35,2198995,709117,Q93.5,HD,1-207.0,2008-07-11 13:18:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,Q93.5,1-207,w,2008,2008,0,0
36,2198995,709117,Q93.5,HD,1-207.0,2008-07-28 11:03:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,Q93.5,1-207,w,2008,2008,0,0
37,2198995,709117,Q93.5,HD,1-207.0,2008-08-26 13:09:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,Q93.5,1-207,w,2008,2008,0,0
38,2198995,709117,Q93.5,HD,1-207.0,2008-10-07 16:47:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,Q93.5,1-207,w,2008,2008,0,0
39,2198995,709117,Q93.5,HD,1-207.0,2009-01-08 09:42:00,19,1451275,2008-05-26 22:05:00,2011-02-28 18:00:00,Q93.5,1-207,w,2008,2009,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616924,4120769,635207,M10.94,HD,8-930,2015-06-24 11:46:00,49,662,2015-06-24 11:46:00,2015-06-24 22:48:00,M10.9,8-930,m,1969,2015,46,5
1616928,4122176,115102,I48.0,HD,8-930,2015-06-27 01:54:00,11,1881,2015-06-27 01:54:00,2015-06-28 09:15:00,I48.0,8-930,m,1941,2015,74,7
1616929,4122371,438450,J45.9,HD,8-930,2015-06-28 02:31:00,41,471,2015-06-28 02:31:00,2015-06-28 10:22:00,J45.9,8-930,m,1982,2015,33,4
1616931,4122526,213945,A09.0,HD,8-930,2015-06-28 22:59:00,19,708,2015-06-28 22:59:00,2015-06-29 10:47:00,A09.0,8-930,m,1932,2015,83,8


In [11]:
# time reduction for date of OPS, admission time and release time, 
# then calculate the relative time in station in day
hdgesamt['OPS_Datum'] = hdgesamt['OPS_Datum'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
hdgesamt['Aufnahmedatum'] = hdgesamt['Aufnahmedatum'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
hdgesamt['Entlassungsdatum'] = hdgesamt['Entlassungsdatum'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
hdgesamt['OPS_Datum_rel'] = hdgesamt['OPS_Datum'] - hdgesamt['Aufnahmedatum']
hdgesamt['OPS_Datum_rel'] = hdgesamt['OPS_Datum_rel'].apply(lambda x: x.days)
hdgesamt['Verweildauer'] = hdgesamt['Entlassungsdatum'] - hdgesamt['Aufnahmedatum']
hdgesamt['Verweildauer'] = hdgesamt['Verweildauer'].apply(lambda x: x.days)

C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\2673516485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdgesamt['OPS_Datum'] = hdgesamt['OPS_Datum'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\2673516485.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdgesamt['Aufnahmedatum'] = hdgesamt['Aufnahmedatum'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
C:\Users\tyto4\AppData\Local\Temp\ipykernel_5224\2673516

In [12]:
# outcome-oriented. Treatment with outcomes death, transferred to hospice or 
# other hospital will be seen as not completed deleted 
hdgesamt = hdgesamt[hdgesamt["Entlassungsgrund"]!=79] # Dead
hdgesamt = hdgesamt[hdgesamt["Entlassungsgrund"]!=119] # To Hospice
hdgesamt = hdgesamt[hdgesamt["Entlassungsgrund"]!=69] # To another hospital
hdgesamt = hdgesamt[hdgesamt["Entlassungsgrund"]!=89] # To another hospital

In [13]:
# hdgesamt_short => column reduction
hdgesamt_short = hdgesamt[["Fall_ID", "Pat_ID", "Geschlecht", "OPS_Altergruppe"	, "ICD","ICDlabel", "OPSlabel", "OPS_Datum_rel", "Verweildauer"]]

In [14]:
# create time indicator "rank" and binning for rank
hdgesamt_short = hdgesamt_short.loc[hdgesamt_short['OPS_Datum_rel']>=0]
bin = np.concatenate([[-100],np.arange(0, 1, 0.01),[100]],axis=0)
label = np.arange(0,101,1)
hdgesamt_short['rank_ratio'] = hdgesamt_short['OPS_Datum_rel']/hdgesamt_short['Verweildauer']
hdgesamt_short['rank_ratio'] = hdgesamt_short['rank_ratio'].fillna(0)
hdgesamt_short['rank'] = pd.cut(x=hdgesamt_short['rank_ratio'], bins=bin,
                     labels=label)

In [15]:
# eliminate nan
hdgesamt_short = hdgesamt_short.dropna()
# check if NaN-Value still in the daten set
#hdgesamt_short[hdgesamt_short['rank'].isna() == True] 

In [16]:
# find min value of rank, drop duplicate
rank_min=hdgesamt_short.groupby(['Fall_ID','OPSlabel'])['rank'].min()
hdgesamt_short = hdgesamt_short.merge(rank_min, on=['Fall_ID','OPSlabel'], how='outer', suffixes=('', '_min'))
hdgesamt_nodup = hdgesamt_short[hdgesamt_short['rank']==hdgesamt_short['rank_min']].drop_duplicates()

In [17]:
# bad pathway eliminated
hdgesamt_nodup['OPS_lvl_1'] = hdgesamt_nodup['OPSlabel'].astype(str).str[0]
allcases = hdgesamt_nodup.groupby(["Fall_ID"], as_index = False)['OPS_lvl_1'].agg(pd.Series.tolist) #as_index is the key ...
allcases['incompleteness'] = allcases['OPS_lvl_1'].apply(lambda x: (str(5) not in x) & (str(6) not in x) & (str(8) not in x) & (str(9) not in x))
completecases = allcases[allcases['incompleteness'] == False] # create a data set of "complete" cases
hdgesamt_nodup=hdgesamt_nodup.merge(completecases, how='inner', on='Fall_ID', 
            left_on=None, right_on=None, left_index=False, right_index=False, 
            sort=False, copy=True, indicator=False, 
            validate=None) # and then merge it with the original
hdgesamt_nodup=hdgesamt_nodup.drop(labels=['OPS_lvl_1_x','OPS_lvl_1_y','incompleteness'], axis=1) # delete unnecessary columns

In [18]:
# LOS (length of stay) for ICD using median
icd_verweil=hdgesamt_nodup.groupby(['ICD'])['Verweildauer'].median()
icd_list=sorted(hdgesamt_nodup.ICD.unique())
hdgesamt_verweil = pd.DataFrame({'ICD':icd_list, 'Verweildauer':icd_verweil})

In [19]:
# save df as csv
hdgesamt.to_csv(index=False,path_or_buf=r"preprocessedData\\hdgesamt.csv")
hdgesamt_nodup.to_csv(index=False,path_or_buf=r"preprocessedData\\hdgesamt_nodup.csv")
hdgesamt_short.to_csv(index=False,path_or_buf=r"preprocessedData\\hdgesamt_short.csv")
hdgesamt_verweil.to_csv(index=False,path_or_buf=r"preprocessedData\\hdgesamt_verweil.csv")